<a href="https://colab.research.google.com/github/figaone/ASSIGNMENT1ME592/blob/main/ASSIGNMENT1ME592.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd